In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
#%matplotlib ipympl
import glob as gb
import csv

from sklearn import manifold

In [13]:
def img_read(data_folders, res, states, file_ext="JPG", exclude=None, include=None):
    
    total = []
    total_f = []
    total_lb = []
    for df in data_folders:
        img_path = gb.glob(df+'/*.'+file_ext ) 
        
        for path in img_path:
            clb = -1
            pl = path.lower()
            
            if exclude is not None and exclude in pl:  
                continue
            
            if include is not None and include not in pl:
                continue
            

            
            for lb in states:
                if lb in pl:
                    clb = states.index(lb)
                    total_lb.append(clb)
                    break
            
            if clb == -1:
                continue
            
            img1 = cv2.imread(path, cv2.IMREAD_GRAYSCALE)#'data/Resized_256_256xBroken_0.JPG', cv2.IMREAD_GRAYSCALE)
            if img1 is None:                   #判断读入的img1是否为空，为空就继续下一轮循环
                continue
            
            res1= cv2.resize(img1,(res,res))              #对图片进行缩放，第一个参数是读入的图片，第二个是制定的缩放大小
            res1 = cv2.equalizeHist(res1)
            
            norm_image = cv2.normalize(res1, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
            

            
            # Fourier Transform
            f = np.fft.fft2(norm_image)
            fshift = np.fft.fftshift(f)
            fres = 20*np.ma.log(np.abs(fshift))
            fres2 = fres.flatten().tolist()   
            
            res1_1 = norm_image.flatten() #res1_1 = res1.reshape(1,784)/255.0       #将表示图片的二维矩阵转换成一维
            #im_data = np.concatenate((im_data, res1_1))
            res1_1_1 = res1_1.tolist()                     #将numpy.narray类型的矩阵转换成list
            total.append(res1_1_1)  
            total_f.append(fres2)
    
    im_data = np.array(total).astype('float32')
    im_dataf = np.array(total_f).astype('float32')
    im_lb = np.array(total_lb).astype('int32')
    
    #print(im_dataf.shape)
    im_dataf[np.isnan(im_dataf)] = np.nanmean(im_dataf)
    im_dataf[np.isinf(im_dataf)] = np.nanmean(im_dataf)
    
    data = dict()
    data['spatial'] = im_data
    data['freq'] = im_dataf
    data['label'] = im_lb
    return data


In [14]:
# Preprocess IMG Data
fd = ['data/Paritial/']
res = 64
data = img_read(fd, res, ['dripping', 'jetting', 'wetting', 'broken'])

In [8]:
im_data.tofile('data/pimg'+str(res)+'.bin')
im_dataf.tofile('data/pimgf'+str(res)+'.bin')
im_lb.tofile('data/plabel.bin')

In [27]:
def save_data(name, datas, meta):
    file = open(meta,"w") 
    for data in datas:
        file.write(name+data+'.bin')
        file.write('%')
        file.write('{}'.format(datas[data].shape))
        file.write('%')
        file.write('{}'.format(datas[data].dtype))
        file.write('\n')
        datas[data].tofile(name+data+'.bin')
    file.close()
    return

def load_data(fd, res, meta):
    return

In [29]:
save_data('data/part', data, 'data/meta')